In [ ]:
# Google Colab Optimized PDF Processing with Threading
# This replaces your original sequential processing with parallel threading

!pip install fpdf pymupdf ipywidgets psutil --quiet

from google.colab import files
import zipfile, fitz, pandas as pd
from fpdf import FPDF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import base64
import time
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import queue
import io
import gc
import psutil
import os
from typing import Dict, List, Tuple, Optional
import json

class CoLabPDFProcessor:
    """Optimized PDF processor for Google Colab"""

    def __init__(self, zip_path: str):
        self.zip_path = zip_path
        self.results_queue = queue.Queue()
        self.progress_queue = queue.Queue()
        self.failed_files = []
        self.processed_count = 0
        self.start_time = None

    def get_system_info(self):
        """Get Colab system information"""
        cpu_count = os.cpu_count()
        memory_gb = psutil.virtual_memory().total / (1024**3)
        return {
            'cpu_cores': cpu_count,
            'memory_gb': round(memory_gb, 1),
            'recommended_workers': min(cpu_count * 2, 12)  # Cap at 12 for Colab
        }

    def extract_features(self, text: str) -> Dict:
        """Extract text features with error handling"""
        if not text or not text.strip():
            return {
                "word_count": 0,
                "char_count": 0,
                "line_count": 1,
                "avg_word_length": 0
            }

        words = text.split()
        return {
            "word_count": len(words),
            "char_count": len(text),
            "line_count": max(1, text.count('\n') + 1),
            "avg_word_length": sum(len(w) for w in words) / max(1, len(words))
        }

    def extract_text_safe(self, pdf_data: bytes, filename: str = "") -> str:
        """Safely extract text from PDF bytes"""
        try:
            doc = fitz.open("pdf", pdf_data)
            text = ""
            for page in doc:
                text += page.get_text()
            doc.close()
            return text
        except Exception as e:
            print(f"⚠️ PDF parsing error for {filename}: {e}")
            return ""

    def does_fit_one_page(self, content: str, filename: str = "") -> int:
        """Check if content fits one page using FPDF simulation"""
        if not content.strip():
            return 1

        try:
            pdf = FPDF()
            pdf.add_page()
            pdf.set_font("Arial", size=12)
            # Handle encoding issues
            safe_content = content.encode('latin-1', 'ignore').decode('latin-1')
            pdf.multi_cell(0, 10, safe_content)
            result = int(pdf.page_no() == 1)
            return result
        except Exception as e:
            # Fallback: estimate based on character count
            return int(len(content) < 2000)

    def process_single_pdf(self, pdf_info: Tuple[str, bytes]) -> Optional[Dict]:
        """Process a single PDF file"""
        pdf_name, pdf_data = pdf_info

        try:
            # Skip extremely large files to avoid memory issues
            if len(pdf_data) > 15 * 1024 * 1024:  # 15MB limit for Colab
                self.failed_files.append(f"{pdf_name} (too large: {len(pdf_data)/(1024*1024):.1f}MB)")
                return None

            # Extract text
            text = self.extract_text_safe(pdf_data, pdf_name)

            # Extract features
            features = self.extract_features(text)

            # Check if fits one page
            fits_one_page = self.does_fit_one_page(text, pdf_name)

            # Combine results
            result = {
                'pdf_name': pdf_name,
                'word_count': features['word_count'],
                'char_count': features['char_count'],
                'line_count': features['line_count'],
                'avg_word_length': features['avg_word_length'],
                'label': fits_one_page,
                'file_size_mb': len(pdf_data) / (1024 * 1024),
                'thread_id': threading.current_thread().name
            }

            # Update progress
            self.processed_count += 1
            self.progress_queue.put(self.processed_count)

            return result

        except Exception as e:
            error_msg = f"{pdf_name}: {str(e)}"
            self.failed_files.append(error_msg)
            return None

    def progress_monitor(self, total_files: int, update_interval: float = 2.0):
        """Monitor and display progress in a separate thread"""
        last_update = time.time()

        while self.processed_count < total_files:
            try:
                # Non-blocking check for progress updates
                current_count = self.progress_queue.get_nowait()

                # Update display every few seconds
                current_time = time.time()
                if current_time - last_update >= update_interval:
                    elapsed = current_time - self.start_time
                    rate = current_count / elapsed if elapsed > 0 else 0
                    remaining = (total_files - current_count) / rate if rate > 0 else 0

                    # Memory usage
                    memory_percent = psutil.virtual_memory().percent

                    print(f"\r🔄 Progress: {current_count}/{total_files} "
                          f"({current_count/total_files*100:.1f}%) | "
                          f"Rate: {rate:.1f} files/sec | "
                          f"ETA: {remaining/60:.1f}min | "
                          f"Memory: {memory_percent:.1f}%",
                          end="", flush=True)

                    last_update = current_time

            except queue.Empty:
                time.sleep(0.5)
                continue
            except Exception as e:
                break

        print()  # New line when done

    def process_all_pdfs_threaded(self, max_workers: int = None) -> pd.DataFrame:
        """Process all PDFs using ThreadPoolExecutor"""

        # Get system info and determine optimal workers
        sys_info = self.get_system_info()
        if max_workers is None:
            max_workers = sys_info['recommended_workers']

        print(f"🖥️ System Info: {sys_info['cpu_cores']} cores, {sys_info['memory_gb']}GB RAM")
        print(f"🔧 Using {max_workers} worker threads")

        # Load PDF files from ZIP
        print("📂 Loading PDF files from ZIP...")
        pdf_data_list = []

        try:
            with zipfile.ZipFile(self.zip_path, 'r') as z:
                pdf_files = [f for f in z.namelist()
                           if f.endswith(".pdf") and not f.startswith("__MACOSX")]

                if not pdf_files:
                    raise ValueError("No PDF files found in ZIP")

                print(f"📚 Found {len(pdf_files)} PDF files")

                # Load PDF data into memory (with size filtering)
                for pdf_name in pdf_files:
                    try:
                        file_info = z.getinfo(pdf_name)
                        if file_info.file_size > 20 * 1024 * 1024:  # Skip >20MB files
                            self.failed_files.append(f"{pdf_name} (too large: {file_info.file_size/(1024*1024):.1f}MB)")
                            continue

                        with z.open(pdf_name) as pdf_file:
                            pdf_data = pdf_file.read()
                            pdf_data_list.append((pdf_name, pdf_data))

                    except Exception as e:
                        self.failed_files.append(f"{pdf_name}: {str(e)}")
                        continue

        except Exception as e:
            raise ValueError(f"Failed to read ZIP file: {e}")

        if not pdf_data_list:
            raise ValueError("No valid PDF files to process")

        print(f"✅ Loaded {len(pdf_data_list)} PDFs for processing")

        # Start processing with threading
        self.start_time = time.time()
        results = []

        # Start progress monitor in background
        progress_thread = threading.Thread(
            target=self.progress_monitor,
            args=(len(pdf_data_list),),
            daemon=True
        )
        progress_thread.start()

        # Process PDFs with ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            future_to_pdf = {
                executor.submit(self.process_single_pdf, pdf_data): pdf_data[0]
                for pdf_data in pdf_data_list
            }

            # Collect results as they complete
            for future in as_completed(future_to_pdf):
                pdf_name = future_to_pdf[future]
                try:
                    result = future.result()
                    if result is not None:
                        results.append(result)
                except Exception as e:
                    self.failed_files.append(f"{pdf_name}: {str(e)}")

        # Final progress update
        processing_time = time.time() - self.start_time

        print(f"\n✅ Threading completed in {processing_time:.2f} seconds")
        print(f"📊 Successfully processed: {len(results)}")
        print(f"❌ Failed files: {len(self.failed_files)}")

        if self.failed_files:
            print("⚠️ First 5 failed files:")
            for failed in self.failed_files[:5]:
                print(f"   - {failed}")
            if len(self.failed_files) > 5:
                print(f"   ... and {len(self.failed_files) - 5} more")

        # Memory cleanup
        del pdf_data_list
        gc.collect()

        return pd.DataFrame(results) if results else pd.DataFrame()

# ---- MAIN EXECUTION PIPELINE ----

# Step 1: Upload ZIP with progress indication
print("📤 Upload a ZIP containing multiple PDFs:")
try:
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("No files uploaded")
    zip_path = next(iter(uploaded))
    print(f"✅ Uploaded: {zip_path}")
except Exception as e:
    print(f"❌ Upload failed: {e}")
    exit()

# Step 2: Initialize processor and get system info
processor = CoLabPDFProcessor(zip_path)
sys_info = processor.get_system_info()

print(f"\n🔧 Colab Environment:")
print(f"   • CPU cores: {sys_info['cpu_cores']}")
print(f"   • Memory: {sys_info['memory_gb']} GB")
print(f"   • Recommended workers: {sys_info['recommended_workers']}")

# Step 3: Process PDFs with threading
print(f"\n🚀 Starting threaded processing...")
print(f"⏱️ Estimated time: 3-8 minutes for 1076 files")

start_total = time.time()

try:
    df = processor.process_all_pdfs_threaded()

    if df.empty:
        print("❌ No PDFs could be processed")
        exit()

except Exception as e:
    print(f"❌ Processing failed: {e}")
    exit()

# Step 4: Save dataset and train model
print(f"\n💾 Saving dataset...")
df.to_csv("threaded_pdf_dataset.csv", index=False)
files.download("threaded_pdf_dataset.csv")

print(f"📊 Dataset Summary:")
print(f"   • Total files processed: {len(df)}")
print(f"   • Files that fit (1): {sum(df['label'] == 1)}")
print(f"   • Files that exceed (0): {sum(df['label'] == 0)}")
print(f"   • Average file size: {df['file_size_mb'].mean():.2f} MB")

# Step 5: Train model
print(f"\n🤖 Training RandomForest model...")
X = df.drop(columns=['label', 'pdf_name', 'file_size_mb', 'thread_id'])
y = df['label']

if len(df) < 2:
    X_train, X_test, y_train, y_test = X, X, y, y
    test_accuracy = None
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use more trees since we have threading power
model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test) if len(X_test) > 0 else None

print(f"✅ Model trained:")
print(f"   • Training accuracy: {train_accuracy:.3f}")
if test_accuracy:
    print(f"   • Test accuracy: {test_accuracy:.3f}")

# Step 6: Generate complete results
print(f"\n📋 Generating complete results...")
X_all = df.drop(columns=['label', 'pdf_name', 'file_size_mb', 'thread_id'])
predictions = model.predict(X_all)

complete_results = []
for i, row in df.iterrows():
    complete_results.append({
        'pdf_name': row['pdf_name'],
        'word_count': int(row['word_count']),
        'char_count': int(row['char_count']),
        'line_count': int(row['line_count']),
        'avg_word_length': round(row['avg_word_length'], 2),
        'file_size_mb': round(row['file_size_mb'], 3),
        'actual_label': int(row['label']),
        'predicted_label': int(predictions[i]),
        'actual_result': 'Fits' if row['label'] == 1 else 'Exceeds',
        'predicted_result': 'Fits' if predictions[i] == 1 else 'Exceeds',
        'prediction_correct': 'Yes' if row['label'] == predictions[i] else 'No',
        'processed_by_thread': row['thread_id']
    })

results_df = pd.DataFrame(complete_results)
results_df.to_csv("threaded_pdf_results.csv", index=False)
files.download("threaded_pdf_results.csv")

# Final summary
total_time = time.time() - start_total
accuracy = sum(results_df['prediction_correct'] == 'Yes') / len(results_df)

print(f"\n🎉 THREADED PROCESSING COMPLETE!")
print(f"⏱️ Total time: {total_time/60:.1f} minutes (vs 15-25 min sequential)")
print(f"⚡ Speed improvement: {15*60/total_time:.1f}x faster")
print(f"📁 Results: threaded_pdf_results.csv")
print(f"📊 Final accuracy: {accuracy:.2%}")
print(f"🔧 Files processed by {sys_info['recommended_workers']} threads")

# Enhanced Interactive UI with threading info
pdf_input = widgets.Text(
    placeholder='Type exact PDF name',
    description='📄 PDF Name:',
    layout=widgets.Layout(width='70%')
)

predict_button = widgets.Button(description="🔍 Predict", button_style='primary')
show_threads_button = widgets.Button(description="🧵 Show Thread Stats", button_style='info')
show_all_button = widgets.Button(description="📚 Show All Results", button_style='success')

display(pdf_input, predict_button, show_threads_button, show_all_button)

output_area = widgets.Output()
display(output_area)

# Threading statistics
def show_thread_stats(b):
    with output_area:
        output_area.clear_output()

        thread_stats = df.groupby('thread_id').agg({
            'pdf_name': 'count',
            'file_size_mb': ['mean', 'sum'],
            'label': 'mean'
        }).round(3)

        print("🧵 THREADING PERFORMANCE STATISTICS")
        print("=" * 50)
        print(f"Total threads used: {df['thread_id'].nunique()}")
        print(f"Average files per thread: {len(df) / df['thread_id'].nunique():.1f}")
        print(f"Processing time: {total_time:.1f} seconds")
        print(f"Throughput: {len(df) / total_time:.1f} files/second")
        print("\nPer-thread breakdown:")
        print(thread_stats)

def predict_single(b):
    with output_area:
        output_area.clear_output(wait=True)

        pdf_name = pdf_input.value.strip()
        if not pdf_name:
            print("❌ Please enter a PDF name")
            return

        # Find in results
        match = results_df[results_df['pdf_name'].str.contains(pdf_name, case=False, na=False)]

        if match.empty:
            print(f"❌ '{pdf_name}' not found in processed results")
            return

        row = match.iloc[0]
        print(f"📘 PDF: {row['pdf_name']}")
        print(f"📏 Features: {row['word_count']} words, {row['line_count']} lines")
        print(f"📦 File size: {row['file_size_mb']} MB")
        print(f"🤖 Model prediction: {'✅ Fits' if row['predicted_label'] == 1 else '❌ Exceeds'}")
        print(f"🧾 Actual result: {'✅ Fits' if row['actual_label'] == 1 else '❌ Exceeds'}")
        print(f"🧵 Processed by: {row['processed_by_thread']}")
        print(f"✅ Prediction {'correct' if row['prediction_correct'] == 'Yes' else 'incorrect'}")

def show_all_results(b):
    with output_area:
        output_area.clear_output()

        print(f"📊 COMPLETE RESULTS SUMMARY")
        print(f"Total processed: {len(results_df)}")
        print(f"Accuracy: {accuracy:.2%}")
        print(f"Processing speed: {len(results_df)/total_time:.1f} files/sec")
        print("\nFirst 10 results:")

        display_cols = ['pdf_name', 'word_count', 'predicted_result', 'actual_result', 'prediction_correct']
        display(results_df[display_cols].head(10))

predict_button.on_click(predict_single)
show_threads_button.on_click(show_thread_stats)
show_all_button.on_click(show_all_results)

print(f"\n🎯 Ready! Your {len(results_df)} PDFs processed in {total_time/60:.1f} minutes using threading!")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.7 MB/s eta 0:00:00
📤 Upload a ZIP containing multiple PDFs:
